## Add Kernel Prep

This is a small section to modify RAJAPerf source to add a new kernel

[Back to Table of Contents](./00-intro-and-contents.ipynb)

In [ ]:
%%bash
# Run this cell to perform some setup to modify the RAJAPerf source to accommodate the new kernel
pushd $HOME/scripts
chmod +x modify_rajaperf_files.sh
chmod +x restore_rajaperf_files.sh
./modify_rajaperf_files.sh
popd
# note you can restore the state of RAJAPerf by exiting to a terminal session
# pushd to ~/scripts directory
# run restore_rajaperf_files.sh
# or add a new %%bash cell below and do the same

## Script details 
1: Edit src/CMakeLists.txt to add another directory dependency
```
include_directories(.)

add_subdirectory(tutkernel)

set(RAJA_PERFSUITE_EXECUTABLE_DEPENDS
    common
    apps
    basic
    basic-kokkos
    lcals
    lcals-kokkos
    polybench
    stream
    stream-kokkos
    algorithm)
list(APPEND RAJA_PERFSUITE_EXECUTABLE_DEPENDS ${RAJA_PERFSUITE_DEPENDS} tutkernel)
```

2: Add group Tutorial to RAJAPerfSuite.hpp GroupID
```
enum GroupID {

  Basic = 0,
  Lcals,
  Polybench,
  Stream,
  Apps,
  Algorithm,
  Tutorial,

  NumGroups // Keep this one last and DO NOT remove (!!)

};
```

3: Add Tutorial_KERNEL to RAJAPerfSuite.hpp KernelID
```
enum KernelID {
...
  Algorithm_MEMCPY,
//
// Tutorial Kernel
//
  Tutorial_KERNEL,

  NumKernels // Keep this one last and NEVER comment out (!!)
};
```

4: Add tutkernel/TUTORIAL_KERNEL.hpp header to RAJAPerfSuite.cpp

5: Add group string ("Tutorial") to RAJAPerfSuite.cpp to line up with our GroupID enum
```
static const std::string GroupNames [] =
{
  std::string("Basic"),
  std::string("Lcals"),
  std::string("Polybench"),
  std::string("Stream"),
  std::string("Apps"),
  std::string("Algorithm"),
  std::string("Tutorial"),

  std::string("Unknown Group")  // Keep this at the end and DO NOT remove....

}; // END GroupNames

```

6: Add kernel string ("Tutorial_KERNEL") to line up with  KernelID enum
```
static const std::string KernelNames [] =
{
...
  std::string("Algorithm_MEMCPY"),

//
// Tutorial Kernel
//
  std::string("Tutorial_KERNEL"),

  std::string("Unknown Kernel")  // Keep this at the end and DO NOT remove....

}; // END KernelNames
```

7: add case Tutorial_KERNEL to instantiate the new kernel class 
```
KernelBase* getKernelObject(KernelID kid,
                            const RunParams& run_params)
{
  KernelBase* kernel = 0;

  switch ( kid ) {
  
  ...
   case Algorithm_MEMCPY: {
       kernel = new algorithm::MEMCPY(run_params);
       break;
    }

    
//
// Tutorial Kernel
//
    case Tutorial_KERNEL: {
      kernel = new tutorial::TUTORIAL_KERNEL(run_params);
      break;
    }

    default: {
      getCout() << "\n Unknown Kernel ID = " << kid << std::endl;
    }

  } // end switch on kernel id

  return kernel;
}  
```